<a href="https://colab.research.google.com/github/sw-programmer/Coursework1/blob/sangwoo/CW1_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **Coursework 1**

1.   ### **Computationally Efficient Eigenfaces**
2.   ### **Incremental PCA**
3.   ### **PCA-LDA for Face Recognition**
4.   ### **K-means codebook**
5.   ### **RF classifier**


**by 20190223	Soohyun Ryu,	20190247	Sangwoo Park**






**1. Setting!**

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import io
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, IncrementalPCA

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
mat_file = io.loadmat("/content/drive/MyDrive/CV_ML/CW1/face.mat")
mat_X = mat_file['X']
mat_Y = mat_file['l'][0]

print(mat_file.keys()) 
print(mat_X.shape)                         # 총 52명. 각 10장의 사진. 

dict_keys(['__header__', '__version__', '__globals__', 'X', 'l'])
(2576, 520)


In [72]:
df = pd.DataFrame(data=mat_X).transpose()
df['label'] = mat_Y                         # row -> data, col -> feature
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values                   # Target column

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y, 
                                                    test_size=0.2, 
                                                    shuffle=True,
                                                    stratify=Y, 
                                                    random_state=1000)          
# 얼굴 그룹 비율 유지               
# test/train ratio = 0.2
# random_state -> before dividing, seed for shuffling
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
np.unique(y_train, return_counts=True)    # 52명의 얼굴이 80%씩 고르게 train set에 들어간 모습

X_train shape: (416, 2576)
X_test shape: (104, 2576)
y_train shape: (416,)
y_test shape: (104,)


(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52], dtype=uint8),
 array([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8]))

In [38]:
X_train_lefthalf, X_train_righthalf, y_train_lefthalf, y_trian_righthalf = train_test_split(X_train, 
                                                    y_train, 
                                                    test_size=0.5, 
                                                    shuffle=True,
                                                    stratify=y_train, 
                                                    random_state=35)

# train set 4개로 분리
X_train_1, X_train_2, y_train_1, y_trian_2 = train_test_split(X_train_lefthalf, 
                                                    y_train_lefthalf, 
                                                    test_size=0.5, 
                                                    shuffle=True,
                                                    stratify=y_train_lefthalf, 
                                                    random_state=35)

X_train_3, X_train_4, y_train_3, y_trian_4 = train_test_split(X_train_righthalf, 
                                                    y_trian_righthalf, 
                                                    test_size=0.5, 
                                                    shuffle=True,
                                                    stratify=y_trian_righthalf, 
                                                    random_state=35)

print(np.unique(y_train_3, return_counts=True)) # 인당 2개의 image가 각 data set에 포함

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52], dtype=uint8), array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2]))


In [50]:
# 평균 얼굴 계산 함수 
def AvgCompute(X): 
  Xbar = np.mean(X, axis=0)  
  matrix_A = X - Xbar 
  return matrix_A, Xbar

# 고유값, 고유벡터 계산 함수
def eig(S): 
  eig_vals, eig_vecs = np.linalg.eig(S) 
  sort_indices = np.argsort(eig_vals)[::-1] # 내림차순 정렬 
  return eig_vals[sort_indices], eig_vecs[:, sort_indices]

# Reconstruction에 쓰일 함수
def reconstruct(X, PC):
  return (X @ PC) @ PC.T 

# PCA 과정
def PCA(X, num_components, condition=1): # X == train set, num_components == 선택할 eigen vectoe 개수, condition == 1 -> low, otherwise high
  N, D = X.shape 
  matrix_A, X_bar = AvgCompute(X)
  if condition == 1 : 
    S = (matrix_A @ matrix_A.T) / N
  else: 
    S = (matrix_A.T @ matrix_A) / N

  eig_vals, eig_vecs = eig(S) 
  
  principal_vals, principal_components = np.real(eig_vals[:num_components]), np.real(eig_vecs[:,:num_components]) 
  
  reconst_X = reconstruct(matrix_A, principal_components) + X_bar
  reconst_Err = np.sqrt( np.sum( np.square( reconst_X - X ) ))
  
  Projected_X = X @ principal_components

  return Projected_X, X_bar, principal_components, reconst_Err

### Batch PCA ###

In [73]:
# from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import time
start = time.time()

X_Projected, X_bar, principal_components, reconst_Err = PCA(X_train, 100, 0)

print(f"Training time: {time.time() - start}s")

Training time: 15.634403944015503s


In [74]:
# Reconstruction Error
print(reconst_Err)

10858.835929213785


In [75]:
# NN Classfication
X_result = X_test @ principal_components

y_pred = []
label = 0
for query in X_result:
  Args = []
  for trained in X_Projected:
    Args.append(np.linalg.norm(query - trained))
  y_pred.append(y_train[np.argmin(Args)])

print(accuracy_score(y_test, y_pred))

0.5865384615384616
